In [1]:
import pandas as pd

In [2]:
# We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
# Download the data for January and February 2023.

data = pd.read_parquet("yellow_tripdata_2023-01.parquet")

In [3]:
data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [5]:
# Q1. Downloading the data
# Read the data for January. How many columns are there?

data.shape[1]

19

In [6]:
# Q2. Computing duration
# Now let's compute the duration variable. It should contain the duration of a ride in minutes.

duration = (data["tpep_dropoff_datetime"] - data["tpep_pickup_datetime"])
data["duration"] = duration.dt.total_seconds() / 60

# What's the standard deviation of the trips duration in January?

duration.std()/60

Timedelta('0 days 00:00:42.594351')

In [7]:
# Q3. Dropping outliers
# Next, we need to check the distribution of the duration variable. 
# There are some outliers. 
# Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

old = duration.shape[0]
data = data[(data["duration"] >= 1) & (data["duration"] <= 60)]
new = data["duration"].shape[0]

new/old

0.9812202822125979

In [8]:
# Q4. One-hot encoding
# Let's apply one-hot encoding to the pickup and dropoff location IDs. 
# We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)?

data_encoded = data[["PULocationID", "DOLocationID"]]
data_encoded = pd.get_dummies(data_encoded, columns=['PULocationID', 'DOLocationID'])
data_encoded.shape[1]

515

In [9]:
# Q5. Training a model
# Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = data_encoded
y = data["duration"]

model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
mean_squared_error(y, y_pred)**0.5

7.649264406125076

In [13]:
# Q6. Evaluating the model
# Now let's apply this model to the validation dataset (February 2023).

data_val = pd.read_parquet("yellow_tripdata_2023-02.parquet")
duration_val = (data_val["tpep_dropoff_datetime"] - data_val["tpep_pickup_datetime"])
data_val["duration"] = duration_val.dt.total_seconds() / 60
# data_val = data_val[(data_val["duration"] >= 1) & (data_val["duration"] <= 60)]

data_encoded_val = data_val[["PULocationID", "DOLocationID"]]
data_encoded_val = pd.get_dummies(data_encoded_val, columns=['PULocationID', 'DOLocationID'])
missing_cols = set(data_encoded.columns) - set(data_encoded_val.columns)
for c in missing_cols:
    data_encoded_val[c] = 0
data_encoded_val = data_encoded_val[data_encoded.columns]

# What's the RMSE on validation?

X_val = data_encoded_val
y_val = data_val["duration"]
y_pred_val = model.predict(X_val)
mean_squared_error(y_val, y_pred_val)**0.5

3214728.79753912